In [366]:
import numpy as np 
import torch 
import pandas as pd
import yfinance as yf
from arch import arch_model


In [367]:
data = yf.download('^GSPC', start="2015-01-01", end="2025-01-01")

[*********************100%***********************]  1 of 1 completed


In [368]:
data.reset_index(inplace=True)

In [369]:
data = data.Close

In [370]:
data.reset_index(inplace=True)

In [371]:
data['log_returns'] = np.log(data['^GSPC']/data['^GSPC'].shift(-1))
data['volatility'] = data['log_returns'].rolling(window=5).apply(lambda x: (np.sqrt(np.sum(x**2))))
data['volatility'] = data['volatility']*100
data['log_returns'] = data['log_returns']*100
data.drop(['^GSPC','index'], axis=1, inplace=True)
data.fillna(0, inplace=True)

In [372]:
train_len = int(len(data) * 0.8)
val_len = int(len(data)*0.1 + train_len)
test_len = int(len(data)-train_len-val_len)

In [373]:
train_data = data.iloc[:train_len]
val_data = data.iloc[train_len:val_len]
test_data = data.iloc[val_len:int(len(data))]

In [374]:
train_data

Ticker,log_returns,volatility
0,1.844721,0.000000
1,0.893325,0.000000
2,-1.156274,0.000000
3,-1.773017,0.000000
4,0.843932,3.064932
...,...,...
2007,1.455714,2.526099
2008,-0.585095,2.338602
2009,0.405784,2.194310
2010,1.209347,2.503352


In [375]:
garch = arch_model(train_data['log_returns'], vol='GARCH', p=1,q=1, mean='Zero')
garch_fit = garch.fit()

Iteration:      1,   Func. Count:      5,   Neg. LLF: 11170.316194782295
Iteration:      2,   Func. Count:     15,   Neg. LLF: 4643.9877591811355
Iteration:      3,   Func. Count:     22,   Neg. LLF: 5106.334017311465
Iteration:      4,   Func. Count:     28,   Neg. LLF: 2635.042239528903
Iteration:      5,   Func. Count:     32,   Neg. LLF: 2635.042094583321
Iteration:      6,   Func. Count:     35,   Neg. LLF: 2635.042094584078
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2635.042094583321
            Iterations: 6
            Function evaluations: 35
            Gradient evaluations: 6


In [376]:
garch_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                       Zero Mean - GARCH Model Results                        
==============================================================================
Dep. Variable:            log_returns   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2635.04
Distribution:                  Normal   AIC:                           5276.08
Method:            Maximum Likelihood   BIC:                           5292.90
                                        No. Observations:                 2012
Date:                Wed, Mar 12 2025   Df Residuals:                     2012
Time:                        13:47:08   Df Model:                            0
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0385  1.101e-02      3.497  4.697e-04 [1.693e-02,6.010e-02]
alpha[1]       0.1960  3.265e-02      6.001  1.957e-09     [  0.132,  0.260]
beta[1]        0.7795  3.050e-02     25.556 4.740e-144     [  0.720,  0.839]
============================================================================

Covariance estimator: robust
"""

In [377]:
def generate_ground_garch(omega, alpha, beta, n=1000):
    am = arch_model(None, mean='Zero', vol='GARCH', p=1,q=1)

    params = np.array([omega,alpha,beta])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

In [378]:
def generate_ground_gjrgarch(omega, alpha, lmbda, beta, n = 1000):
    am = arch_model(None, mean='Zero', p =1, q = 1, o =1)
    params = np.array([omega,alpha,lmbda,beta])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

In [379]:
def generate_ground_figarch(omega, beta, phi ,d, n = 1000):
    am = arch_model(None, mean='Zero', vol='FIGARCH')
    params= np.array([omega, beta, phi, d])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

In [380]:
generate_ground_garch(0.5,0.2,0.1);

In [381]:
generate_ground_gjrgarch(0.1,0.2,0.3,0.4);

In [382]:
generate_ground_figarch(0.1,0.2,0.3,0.4);

In [ ]:
def create_windows(X, y, window_length, horizon): 
    xs = []
    ys = []

    for i in range(len(y) - window_length - horizon):
        xs.append(X[i:i + window_length])
        ys.append(y[i + window_length + horizon])


    return np.array(xs),np.array(ys).reshape()

In [384]:
X,y = generate_ground_garch(0.1,0.2,0.3)

In [385]:
test_x, test_y = create_windows(X,y,window_length=5, horizon=1)

In [386]:
test_y

array([0.3931652 , 0.59567778, 0.7288381 , 0.55431642, 0.77993003,
       0.5362099 , 0.43203836, 0.39651141, 0.38376652, 0.38896667,
       0.3858009 , 0.38326344, 0.38446694, 0.47574411, 0.42276175,
       0.39573811, 0.38711718, 0.40201236, 0.42759157, 0.39398856,
       0.38751755, 0.39332178, 0.4861566 , 0.41346678, 0.42189868,
       0.52354914, 0.44164707, 0.46322867, 0.42263566, 0.4919082 ,
       0.62390184, 0.56947961, 0.46747294, 0.56019483, 0.44149654,
       0.41744709, 0.39566546, 0.38865282, 0.41558668, 0.50526839,
       0.79429276, 0.59182375, 0.70221669, 0.71373133, 0.52877622,
       0.76408744, 0.74641725, 0.51705704, 0.64196802, 0.47437728,
       0.4578431 , 0.40824653, 0.42825116, 0.49201568, 0.47296009,
       0.43439171, 0.39787195, 0.41234699, 0.42636778, 0.39319659,
       0.48071863, 0.411827  , 0.39781505, 0.39249151, 0.39245225,
       0.43612909, 0.42846241, 0.49532488, 0.72558664, 0.69669522,
       0.69199858, 0.55559828, 0.45260546, 0.40702728, 0.39903

In [387]:
class RNNGARCH (torch.nn.Module): 
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)